# IMDbのDataLoaderの作成

## IMDbデータセットをtsv形式に変換

In [1]:
import glob
import os
import io
import string


# 訓練データのtsvファイルを作成
f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消す
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消す
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

In [2]:
# テストデータの作成
f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消す
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消す
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()

## 前処理と単語分割の関数を定義

In [3]:
import string
import re

# 以下の記号はスペースに置き換える（カンマ、ピリオドを除く）
# punctuationとは日本語で句点という意味
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れる
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）
def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認
print(tokenizer_with_preprocessing('I like cats.'))

区切り文字： !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


## DataLoaderの作成

In [4]:
import torchtext


# 文章とラベルの両方に用意します
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [5]:
# Datasetの作成
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))

訓練および検証のデータ数 25000
1つ目の訓練および検証のデータ {'Text': ['this', 'miracle', 'of', 'a', 'movie', 'is', 'one', 'of', 'those', 'films', 'that', 'has', 'a', 'lasting', ',', 'long', 'term', 'effect', 'on', 'you', '.', 'i', 've', 'read', 'a', 'review', 'or', 'two', 'from', 'angry', 'people', 'who', 'i', 'guess', 'are', 'either', 'republicans', 'or', 'child', 'beaters', ',', 'and', 'their', 'extremist', 'remarks', 'speak', 'of', 'the', 'films', 'power', 'to', 'confront', 'people', 'with', 'their', 'own', 'darkest', 'secrets', '.', 'no', 'such', 'piece', 'of', 'art', 'has', 'ever', 'combined', 'laughter', 'and', 'tears', 'in', 'me', 'before', 'and', 'that', 'is', 'the', 'miracle', 'of', 'the', 'movie', '.', 'the', 'realism', 'of', 'the', 'movie', 'and', 'it', 's', 'performance', 'by', 'bret', 'carr', 'is', 'not', 'to', 'be', 'missed', '.', 'the', 'very', 'nature', 'of', 'it', 's', 'almost', 'interactive', 'effect', ',', 'will', 'cause', 'people', 'to', 'leave', 'the', 'theater', 'either', 'liberated', 'or

In [6]:
import random

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))

訓練データの数 20000
検証データの数 5000
1つ目の訓練データ {'Text': ['suggesting', 'nothing', 'less', 'than', 'a', 'movie', 'length', 'version', 'of', 'the', '1970s', 'tv', 'hit', 'love', ',', 'american', 'style', ',', 'decked', 'out', 'with', 'flashes', 'of', 'nudity', ',', 'superchick', '1973', 'is', 'a', 'lighthearted', 'piece', 'of', 'fluff', 'that', 'somehow', 'still', 'manages', 'to', 'entertain', '.', 'and', 'the', 'lead', 'character', 'here', ',', 'tara', 'b', '.', 'true', ',', 'really', 'is', 'some', 'kind', 'of', 'superchick', '.', 'a', 'stewardess', 'not', 'flight', 'attendant', 'who', 's', 'so', 'good', 'looking', 'that', 'even', 'her', 'plane', 's', 'autopilot', 'has', 'made', 'a', 'pass', 'at', 'her', ',', 'and', 'with', 'a', 'hunky', 'boyfriend', 'in', 'every', 'port', ',', 'this', 'wingin', ',', 'swingin', 'gal', 'really', 'does', 'put', 'the', 'lay', 'in', 'layover', '.', 'what', 'with', 'her', 'germaphobe', 'surgeon', 'beau', 'in', 'new', 'york', ',', 'her', 'playboy', 'with', 'gangster', 

## ボキャブラリーの作成

In [7]:
# torchtextで単語ベクトルとして英語学習済みモデルを読み込む
from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')

# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))

1単語を表現する次元数： 300
単語数： 999994


In [8]:
# ベクトル化したバージョンのボキャブラリーを作成
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認
TEXT.vocab.stoi

torch.Size([17934, 300])


78,
             'sense': 279,
             'ending': 280,
             'worth': 281,
             'dvd': 282,
             'money': 283,
             'true': 284,
             'job': 285,
             'actor': 286,
             'reason': 287,
             'everything': 288,
             'let': 289,
             'someone': 290,
             'maybe': 291,
             'once': 292,
             'wasn': 293,
             'together': 294,
             'shows': 295,
             'play': 296,
             'three': 297,
             'american': 298,
             'watched': 299,
             'house': 300,
             'main': 301,
             'everyone': 302,
             '1': 303,
             'john': 304,
             'effects': 305,
             'audience': 306,
             'version': 307,
             'said': 308,
             'beautiful': 309,
             'instead': 310,
             'high': 311,
             'later': 312,
             'during': 313,
             'night': 314,
        

In [9]:
# DataLoaderを作成（torchtextの文脈では単純にiteraterと呼ぶ）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)

# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)

(tensor([[   2, 4151,   13,  ...,    1,    1,    1],
        [   2,   14, 2217,  ...,    5,    5,    3],
        [   2,  152,   15,  ...,    1,    1,    1],
        ...,
        [   2,    0,  217,  ...,   76, 2897,    3],
        [   2,   12,   17,  ...,    1,    1,    1],
        [   2,    4,  854,  ...,  725,  914,    3]]), tensor([239, 256, 192, 197, 256, 174, 256, 123, 150, 124, 256, 106, 256, 131,
        256, 256,  85, 256, 256, 160, 256, 256, 239, 256]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])
